<a href="https://colab.research.google.com/github/Khyojae/2025.LLM/blob/main/CNN1114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input
from tensorflow.keras.datasets import cifar10

In [4]:


IMG_SIZE = (224,224)
NUM_CLASSES=10
INPUT_SHAPE = (32,32,3)

(x_train,y_train),(x_test,y_test) =cifar10.load_data()

# 원-핫 인코딩
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

# x_train 데이터를 MobileNetV2 권장 전처리 함수와 함께 사용할 수 있도록 float32로 변환
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False

inputs = Input(shape=(32, 32, 3))

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, UpSampling2D
resized = tf.keras.layers.UpSampling2D(size=(7, 7), interpolation='bilinear')(inputs)

preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(resized)
x = base_model(preprocessed, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=inputs, outputs=preds)

train_datagen =ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10
)

train_gen = train_datagen.flow(x_train, y_train, batch_size=16)

for layer in base_model.layers:
  layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

model.fit(train_gen,
          steps_per_epoch=len(x_train)//16,
          epochs=10)



for layer in base_model.layers[-10:]:
  layer.trainable= True

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n# 8. Fine-tuning 학습 시작 (일부 백본 재학습)")
model.fit(
    train_gen,
    # ⭐ 오류 수정: len(16) 대신 len(x_train) // 16 사용
    steps_per_epoch=len(x_train) // 16,
    epochs=10
)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1746s 557ms/step - accuracy: 0.7183 - loss: 0.8188
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1755s 555ms/step - accuracy: 0.8106 - loss: 0.5451
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1752s 551ms/step - accuracy: 0.8328 - loss: 0.4876
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1745s 558ms/step - accuracy: 0.8425 - loss: 0.4531
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1746s 559ms/step - accuracy: 0.8501 - loss: 0.4277
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1737s 556ms/step - accuracy: 0.8647 - loss: 0.3890
Epoch 7/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1774s 560ms/step - accuracy: 0.8739 - loss: 0.3668
Epoch 8/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1761s 559ms/step - accuracy: 0.8818 - loss: 0.3376
Epoch 9/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1756s 562ms/step - accuracy: 0.8895 - loss: 0.3160
Epoch 10/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1753s 559ms/step - accuracy: 0.8944 - loss: 0.3029

# 8. Fine-tuning 학습 시작 (일부 백본 재학습)
Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 21

In [8]:
print("\n# 9. 최종 모델 평가")
res = model.evaluate(x_test, y_test, verbose=1)
print(f"정확률은 {res[1]*100:.2f}% 입니다.")


# 9. 최종 모델 평가
313/313 ━━━━━━━━━━━━━━━━━━━━ 380s 1s/step - accuracy: 0.8842 - loss: 0.3574
정확률은 88.25% 입니다.
